# Preparing Building Electricity consumption data
---

### Overview
1. Inspect raw data
2. Process all data
---

We start with importing hyper parameters and modules

In [1]:
import hyper_buildingelectricity
import prep_building_electricity
import pandas as pd

HYPER = hyper_buildingelectricity.HyperBuildingElectricity()

### 1. Inspect raw data

In [ ]:
df_consumption, df_building_images, df_meteo_dict = prep_building_electricity.import_all_data(HYPER)

display(df_consumption)
display(df_building_images)
display(df_meteo_dict['meteo_1_2014.csv'])

### 2. Process all data

In [ ]:
df_consumption_new, df_building_images_new, df_meteo_dict_new = prep_building_electricity.process_all_data(
    HYPER, 
    df_consumption, 
    df_building_images, 
    df_meteo_dict
)

display(df_consumption_new)
display(df_building_images_new)
display(df_meteo_dict_new)

In [ ]:
(
    df_consumption, 
    df_building_images, 
    df_meteo_dict 
)= prep_building_electricity.process_all_data(
    HYPER, 
    df_consumption, 
    df_building_images, 
    df_meteo_dict
)

display(df_consumption)
display(df_building_images)
#display(df_sample_meteo_data)

In [6]:
# drop the year entries
df_consumption.drop(index=1, inplace=True)

In [9]:
df_consumption

,building ID,1,2,3,4,5,6,7,8,9,...,450,451,452,453,454,455,456,457,458,459
0,cluster ID,32.000000,48.000000,57.000000,76.000000,11.000000,11.000000,11.000000,24.000000,11.000000,...,16.000000,44.000000,63.000000,16.000000,53.000000,16.000000,44.000000,63.000000,83.000000,73.000000
2,2014-01-01 00:15:00,0.561772,16.180502,0.649652,0.650358,2.351030,0.675435,0.073020,0.478699,0.539064,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,2014-01-01 00:30:00,0.842658,15.056856,0.433101,0.789721,3.920245,1.004783,0.074243,0.390285,0.542180,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,2014-01-01 00:45:00,0.561772,14.794672,0.697774,0.789721,3.861395,0.268303,0.066085,0.356182,0.405077,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,2014-01-01 01:00:00,0.842658,14.682308,0.553407,0.650358,3.855435,0.065476,0.076283,0.440807,0.461165,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35037,2014-12-31 23:00:00,0.561772,18.296702,3.344504,0.464542,0.158300,0.994937,0.072204,0.899296,0.369244,...,1.218254,0.204380,0.458663,0.012286,0.708488,1.494477,0.051873,0.287947,0.040328,0.431560
35038,2014-12-31 23:15:00,0.561772,15.562497,0.866202,0.510996,3.339937,0.708419,0.127275,4.499009,0.504789,...,0.346095,0.218005,0.458663,1.167134,0.273734,0.942170,0.074509,0.267380,0.000000,0.503487
35039,2014-12-31 23:30:00,0.561772,15.637407,0.384979,0.371633,3.901994,0.054153,0.158685,4.832456,0.384823,...,0.083063,0.190754,0.393140,1.461988,0.491111,0.812216,0.068850,0.226244,0.040328,0.431560
35040,2014-12-31 23:45:00,0.842658,15.431405,0.264673,0.371633,3.934771,0.070891,0.076283,5.109065,0.433121,...,0.083063,0.218005,0.327616,1.461988,0.144918,0.730994,0.034896,0.329083,0.120983,0.071927


In [50]:
HYPER.PREDICTION_WINDOW = 96

In [85]:

# get corresponding time stamps series and reset indices
time_stamps = df_consumption['building ID'].iloc[1:].reset_index(drop=True)

for building_id in building_id_list:
    
    # get cluster id as integer
    cluster_id = df_consumption[building_id].iloc[0].astype(int)
    
    # get building load with new indices
    building_load = df_consumption[building_id].iloc[1:].reset_index(drop=True)
    
    # transform building id into integer
    building_id = int(building_id)
    
    # determine where iteration ends
    iter_end = len(time_stamps)- HYPER.PREDICTION_WINDOW
    
    # iterate over all time stamps in prediction window steps
    for i in range(0, iter_end, HYPER.PREDICTION_WINDOW):
        
        # get iterated load profile
        load_profile = building_load[i:(i+HYPER.PREDICTION_WINDOW)].values
    
        # get time stamp
        time = time_stamps[i]
        
        # get single entries of timestamp
        year = int(time[0:4])
        month = int(time[5:7])
        day = int(time[8:10])
        hour = int(time[11:13])
        quarter_hour = int(time[14:16])
        
        # create entry values in desired format
        entry = [time, year, month, day, hour, quarter_hour, building_id, cluster_id]
        entry+= list(load_profile)
        
        
        df_consumption_new.loc[1]

In [86]:
entry

['2014-12-30 00:15:00',
 2014,
 12,
 30,
 0,
 15,
 459,
 73,
 0.3596334865684711,
 0.2157800919410826,
 0.1438533946273884,
 0.2517434405979298,
 1.0789004597054133,
 0.2157800919410826,
 0.1078900459705413,
 0.1078900459705413,
 0.0359633486568471,
 0.0,
 0.2157800919410826,
 0.0719266973136942,
 0.1078900459705413,
 0.1078900459705413,
 0.1438533946273884,
 0.0359633486568471,
 0.0719266973136942,
 0.0359633486568471,
 0.0359633486568471,
 0.1078900459705413,
 0.1078900459705413,
 0.1078900459705413,
 0.0719266973136942,
 0.0719266973136942,
 0.0359633486568471,
 0.0359633486568471,
 0.0719266973136942,
 0.1078900459705413,
 0.1078900459705413,
 0.1078900459705413,
 0.0359633486568471,
 0.0719266973136942,
 0.0359633486568471,
 0.7552303217937893,
 0.1798167432842355,
 0.1798167432842355,
 0.1798167432842355,
 0.1078900459705413,
 0.1078900459705413,
 0.0719266973136942,
 0.1798167432842355,
 0.3236701379116239,
 0.2157800919410826,
 0.1438533946273884,
 0.1078900459705413,
 0.071926

In [84]:
time[8:10]

'30'

In [ ]:
year 	month 	day 	hour 	quarter_of_hour 	building_id 	cluster_id 	load_1

In [75]:
time[11:13]

'00'

In [77]:
year

2014

In [78]:
month

12

In [79]:
day

30

In [80]:
hour

0

In [81]:
quarter_hour

15

In [66]:
time

'2014-12-30 00:15:00'

In [60]:
load_profile

array([0.35963349, 0.21578009, 0.14385339, 0.25174344, 1.07890046,
       0.21578009, 0.10789005, 0.10789005, 0.03596335, 0.        ,
       0.21578009, 0.0719267 , 0.10789005, 0.10789005, 0.14385339,
       0.03596335, 0.0719267 , 0.03596335, 0.03596335, 0.10789005,
       0.10789005, 0.10789005, 0.0719267 , 0.0719267 , 0.03596335,
       0.03596335, 0.0719267 , 0.10789005, 0.10789005, 0.10789005,
       0.03596335, 0.0719267 , 0.03596335, 0.75523032, 0.17981674,
       0.17981674, 0.17981674, 0.10789005, 0.10789005, 0.0719267 ,
       0.17981674, 0.32367014, 0.21578009, 0.14385339, 0.10789005,
       0.0719267 , 0.0719267 , 0.0719267 , 0.17981674, 1.47449729,
       0.17981674, 0.32367014, 0.21578009, 0.25174344, 0.35963349,
       0.39559684, 0.28770679, 0.32367014, 0.35963349, 0.39559684,
       0.39559684, 0.43156018, 0.35963349, 0.32367014, 0.43156018,
       0.43156018, 0.46752353, 0.61137693, 0.50348688, 0.53945023,
       0.61137693, 0.68330362, 0.68330362, 0.68330362, 0.61137

In [46]:
cluster_id

73

In [47]:
building_load

0        0.000000
1        0.000000
2        0.000000
3        0.000000
4        0.000000
           ...   
35035    0.431560
35036    0.503487
35037    0.431560
35038    0.071927
35039    0.035963
Name: 459, Length: 35040, dtype: float64

In [49]:
time_stamps

0        2014-01-01 00:15:00
1        2014-01-01 00:30:00
2        2014-01-01 00:45:00
3        2014-01-01 01:00:00
4        2014-01-01 01:15:00
                ...         
35035    2014-12-31 23:00:00
35036    2014-12-31 23:15:00
35037    2014-12-31 23:30:00
35038    2014-12-31 23:45:00
35039    2015-01-01 00:00:00
Name: building ID, Length: 35040, dtype: object

In [27]:
cluster_id

73.0

In [17]:
building_id_list = list(df_consumption.columns.values[1:])
building_id_list

['1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '68',
 '69',
 '70',
 '71',
 '72',
 '73',
 '74',
 '75',
 '76',
 '77',
 '78',
 '79',
 '80',
 '81',
 '82',
 '83',
 '84',
 '85',
 '86',
 '87',
 '88',
 '89',
 '90',
 '91',
 '92',
 '93',
 '94',
 '95',
 '96',
 '97',
 '98',
 '99',
 '100',
 '101',
 '102',
 '103',
 '104',
 '105',
 '106',
 '107',
 '108',
 '109',
 '110',
 '111',
 '112',
 '113',
 '114',
 '115',
 '116',
 '117',
 '118',
 '119',
 '120',
 '121',
 '123',
 '124',
 '125',
 '126',
 '127',
 '128',
 '129',
 '130',
 '131',
 '132',
 '133',
 '134',
 '135',
 '136',
 '137',
 '138',
 '139',
 '14

In [88]:
new_df_columns = ['time_stamp', 'year', 'month', 'day', 'hour', 'quarter_of_hour', 'building_id', 'cluster_id']
for pred_time_step in range(96):
    entry_name = 'load_{}'.format(pred_time_step+1)
    new_df_columns.append(entry_name)

df_consumption_new = pd.DataFrame(columns=new_df_columns)
df_consumption_new

,time_stamp,year,month,day,hour,quarter_of_hour,building_id,cluster_id,load_1,load_2,...,load_87,load_88,load_89,load_90,load_91,load_92,load_93,load_94,load_95,load_96


In [98]:
df_consumption_new.loc[1]=entry

In [99]:
df_consumption_new

,time_stamp,year,month,day,hour,quarter_of_hour,building_id,cluster_id,load_1,load_2,...,load_87,load_88,load_89,load_90,load_91,load_92,load_93,load_94,load_95,load_96
1,2014-12-30 00:15:00,2014,12,30,0,15,459,73,0.359633,0.21578,...,0.359633,0.32367,0.287707,0.32367,0.97101,0.53945,0.575414,0.791194,0.611377,0.035963


In [91]:
len(entry)

104

In [92]:
len(df_consumption_new.columns)

104